In [1]:
import os
import numpy as np
from PIL import Image
import copy

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
trial_model=load_model("C:\\Users\\Om\\Desktop\\Cerberus\\Utilities\\VGGFace.h5")

C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [5]:
def feature_extractor(inpDir,personID,wordID,timestep): #access directory and extract face vector     
        
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir=inpDir #input dataset
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[personID]
    personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
    lpersonFolder = os.listdir(personFolder)
    wordID= lpersonFolder[wordID]
    wordFolder = '%s\\%s' % (personFolder,wordID)
    lwordFolder = os.listdir(wordFolder)
    i=0
    for utterance in lwordFolder:
        
        utterFolder= '%s\\%s' % (wordFolder,utterance)
        lutterFolder = os.listdir(utterFolder)
        colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
        lcolourFolder = os.listdir(colourFolder)
        frame = lcolourFolder[timestep]
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True)
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
            
   
    return feature_sequence

In [6]:
people_list_for_training=[1,2,10,11,12]

In [12]:
def people_extractor(inpDir,person_list):
    people_array=[]
    dim_count=0
    for people in range(0,len(person_list)):
        feature_sequence=feature_extractor(inpDir,person_list[people],0,0)
        if dim_count==0:
            people_array=feature_sequence
        else:
            people_array=np.concatenate((people_array,feature_sequence))
        dim_count = dim_count + 1
        print(people_array.shape)
    return people_array

In [13]:
people_array=people_extractor("C:\\Users\\Om\\Desktop\\dataset",people_list_for_training)

(26220,)
(52440,)
(78660,)
(104880,)
(131100,)


In [14]:
people_array.shape

(131100,)

In [15]:
from numpy import save

In [16]:
np.save("Training_array1.npy",people_array)